#LOAD LIBRARIES

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd 
import chardet
import multiprocessing
import random

from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import timeit

#DATASET

##Read Dataset

Let's read the dataset to use.

In [0]:
df = pd.read_csv('/content/drive/My Drive/final_infection_dataset.csv')

In [67]:
df

,fiebre,tos_seca,fatiga,flemas_esputos,dif_resp,dolor_garganta,dolor_cabeza,dolor_muscular,escalofrios,nausea_vomito,cong_nasal,diarrea,expect_sangre,conjuntivitis,dermatitis,falta_gusto_olfato,proteccion_no,proteccion_si,transporte_no,transporte_si,contacto_directo_no,contacto_directo_si,exposicion_alta,exposicion_baja,exposicion_media,zona_andalucia,zona_aragon,zona_asturias,zona_baleares,zona_canarias,zona_cantabria,zona_castilla_la_mancha,zona_castilla_y_leon,zona_cataluña,zona_ceuta,zona_com_val,zona_extremadura,zona_galicia,zona_larioja,zona_madrid,zona_melilla,zona_murcia,zona_navarra,zona_pais_vasco,Prob_infection_initial,Prob_sintomatologia,user_id,Contacto_bluetooth,gestos_de_peligrosidad,tos_detectada,Prob_infection_current,Classification
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,63.9550,0.00,0,46.863905,0.0,0.0,57.328669,Sospechoso_sin_sintomas
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,72.6230,27.95,1,51.289800,0.0,0.0,66.527776,Sospechoso_sin_sintomas
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,71.0180,0.00,2,38.823529,0.0,0.0,58.044324,Sospechoso_sin_sintomas
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,58.6430,27.95,3,38.823529,0.0,0.0,63.351324,Sospechoso_sin_sintomas
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,68.6630,12.11,4,46.863905,0.0,0.0,62.036669,Sospechoso_sin_sintomas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572859,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,60.0990,66.36,1572859,46.863905,0.0,0.0,58.646976,Sospechoso_con_sintomas
1572860,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,40.3120,50.52,1572860,70.588235,0.0,0.0,57.709059,Sospechoso_con_sintomas
1572861,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,54.4675,78.47,1572861,38.823529,0.0,0.0,63.923382,Sospechoso_con_sintomas
1572862,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,48.2330,50.52,1572862,83.054506,0.0,0.0,55.828627,Sospechoso_con_sintomas


#Split in Train/Validación/Test



We will use the usually recommended ratios as an example:

Train: 70%.

Validation: 15%.

Test: 15%.



In [0]:
X=pd.DataFrame()

In [0]:
y = df['Classification']
X = df.loc[:, df.columns != 'Classification']
X = X.loc[:, X.columns != 'tos_seca']
X = X.loc[:, X.columns != 'user_id']
X = X.loc[:, X.columns != 'contacto_directo_no']
X = X.loc[:, X.columns != 'transporte_no']
X = X.loc[:, X.columns != 'proteccion_no']
X = X.loc[:, X.columns != 'Prob_sintomatologia']
X = X.loc[:, X.columns != 'Prob_infection_current']
X = X.loc[:, X.columns != 'Contacto_bluetooth']
X = X.loc[:, X.columns != 'Prob_infection_initial']

In [70]:
X = X.apply(pd.to_numeric) 
X

,fiebre,fatiga,flemas_esputos,dif_resp,dolor_garganta,dolor_cabeza,dolor_muscular,escalofrios,nausea_vomito,cong_nasal,diarrea,expect_sangre,conjuntivitis,dermatitis,falta_gusto_olfato,proteccion_si,transporte_si,contacto_directo_si,exposicion_alta,exposicion_baja,exposicion_media,zona_andalucia,zona_aragon,zona_asturias,zona_baleares,zona_canarias,zona_cantabria,zona_castilla_la_mancha,zona_castilla_y_leon,zona_cataluña,zona_ceuta,zona_com_val,zona_extremadura,zona_galicia,zona_larioja,zona_madrid,zona_melilla,zona_murcia,zona_navarra,zona_pais_vasco,gestos_de_peligrosidad,tos_detectada
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572859,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
1572860,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0
1572861,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
1572862,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0


In [0]:
perc_values = [0.7, 0.15, 0.15];

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train_rand, X_valtest_rand, y_train_rand, y_valtest_rand = train_test_split(X, y, test_size=perc_values[1] + perc_values[2], random_state=1);

X_val_rand, X_test_rand, y_val_rand, y_test_rand = train_test_split(X_valtest_rand, y_valtest_rand, test_size= perc_values[2] / (perc_values[1] + perc_values[2]), random_state=1)

In [74]:
print('Train data size = ' + str(X_train_rand.shape))
print('Train target size = ' + str(y_train_rand.shape))
print('Validation data size = ' + str(X_val_rand.shape))
print('Validation target size = ' + str(y_val_rand.shape))
print('Test data size = ' + str(X_test_rand.shape))
print('Test target size = ' + str(y_test_rand.shape))

Train data size = (1101004, 42)
Train target size = (1101004,)
Validation data size = (235930, 42)
Validation target size = (235930,)
Test data size = (235930, 42)
Test target size = (235930,)


# RANDOM FOREST

1) Import model.

In this case we have to go outside of scikit-learn to apply Random Forest Classifier model.

In [0]:
from sklearn.ensemble import RandomForestClassifier

2) Import metric

In [0]:
from sklearn.metrics import roc_auc_score as auc;
from sklearn.metrics import confusion_matrix as confusion_matrix;
from sklearn.metrics import accuracy_score as acc;

3) Define the method

In [77]:
model = RandomForestClassifier(n_estimators = 50,  random_state = 1)

model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

4) Call the fit method to train the model

In [78]:
start = timeit.default_timer()

model.fit(X_train_rand, np.array(y_train_rand))

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  75.21648618400013


5) Call the predict method to generate the predictions.

In [79]:
start = timeit.default_timer()

pred_train = model.predict(X_train_rand)
pred_val = model.predict(X_val_rand)
pred_test = model.predict(X_test_rand)

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  21.71975921199987


In [0]:


# Label test

lb = preprocessing.LabelBinarizer()
lb.fit(y_test_rand)

y_test_lb = lb.transform(y_test_rand)
val_lb = lb.transform(pred_test)

#Label train

lb.fit(y_train_rand)

y_train_lb = lb.transform(y_train_rand)
val_train_lb = lb.transform(pred_train)

#Label validation

lb.fit(y_val_rand)

y_val_lb = lb.transform(y_val_rand)
val_val_lb = lb.transform(pred_val)


6) Calculate metrics using the predictions obtained in the previous step.

In [0]:
auc_train = auc(y_train_lb, val_train_lb, average='macro');
auc_val = auc(y_val_lb, val_val_lb, average='macro');
auc_test = auc(y_test_lb, val_lb, average='macro');
results = pd.DataFrame()
results = results.append(pd.DataFrame(data={'model':['Random Forest Infection'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test]}, columns=['model',  'auc_train','auc_val', 'auc_test']), ignore_index=True)

In [82]:
results

,model,auc_train,auc_val,auc_test
0,Random Forest Infection,0.995362,0.833785,0.833628


In [0]:
acc_train = acc(y_train_lb, val_train_lb);
acc_val = acc(y_val_lb, val_val_lb);
acc_test = acc(y_test_lb, val_lb);
results2 = pd.DataFrame()
results2 = results2.append(pd.DataFrame(data={'model':['Random Forest Infection'],'acc_train':[acc_train],'acc_val':[acc_val],'acc_test':[acc_test]}, columns=['model',  'acc_train','acc_val', 'acc_test']), ignore_index=True)

In [84]:
results2

,model,acc_train,acc_val,acc_test
0,Random Forest Infection,0.999281,0.976141,0.975934


## Importance of features

In [0]:
importances = list(model.feature_importances_)

In [0]:
# tuplas de importancia y variable
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(X, importances)]
# Ordenamos las variables por importancia
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)


In [87]:
[print(pair) for pair in feature_importances];

('fiebre', 0.494)
('fatiga', 0.104)
('flemas_esputos', 0.087)
('tos_detectada', 0.035)
('dif_resp', 0.032)
('dolor_garganta', 0.026)
('dolor_cabeza', 0.024)
('dolor_muscular', 0.024)
('escalofrios', 0.022)
('nausea_vomito', 0.011)
('cong_nasal', 0.011)
('diarrea', 0.011)
('expect_sangre', 0.01)
('conjuntivitis', 0.01)
('dermatitis', 0.01)
('falta_gusto_olfato', 0.01)
('proteccion_si', 0.008)
('transporte_si', 0.008)
('contacto_directo_si', 0.007)
('exposicion_alta', 0.004)
('exposicion_baja', 0.004)
('exposicion_media', 0.003)
('gestos_de_peligrosidad', 0.003)
('zona_andalucia', 0.002)
('zona_aragon', 0.002)
('zona_asturias', 0.002)
('zona_baleares', 0.002)
('zona_canarias', 0.002)
('zona_cantabria', 0.002)
('zona_castilla_la_mancha', 0.002)
('zona_castilla_y_leon', 0.002)
('zona_cataluña', 0.002)
('zona_ceuta', 0.002)
('zona_com_val', 0.002)
('zona_extremadura', 0.002)
('zona_galicia', 0.002)
('zona_larioja', 0.002)
('zona_madrid', 0.002)
('zona_melilla', 0.002)
('zona_murcia', 0.002)

In [88]:
len(feature_importances)

42